In [161]:
import json
import os
from random import randint

In [162]:
coco_path = 'MASTER.json'
sv_path = 'Screenshot (509).png.json'

In [163]:
# Load coco json
f = open(coco_path)
coco_file = json.load(f)

# Load supervisely json
f2 = open(sv_path)
sv_file = json.load(f2)


In [166]:
output_json = {}

for annot in coco_file['annotations']:
    #print(annot)
    #get image index
    for ind, image in enumerate(coco_file['images']):
        if image['id'] == annot['image_id']:
            image_index = ind
        
    #get image name
    image_name = coco_file['images'][image_index]['file_name']
    
    # create new json in output dict if doesnt exist and do these one time operations
    if image_name not in output_json.keys():
        output_json[image_name] = {}
        output_json[image_name]['tags'] = []
        output_json[image_name]['size'] = {'height': coco_file['images'][image_index]['height'], 'width': coco_file['images'][image_index]['width']}
        output_json[image_name]['objects'] = []

    #annot dict to fill and later append to 'objects' - filled with static values on declaration
    temp_objects_dict = {'id': randint(10000000, 99999999), 'description': '', 
                                            'geometryType': 'polygon', 'labelerLogin': '','createdAt': '',
                                            'updatedAt': '', 'tags': [], 'points': {'exterior': [], 'interior': []}}

    #get class name and id
    temp_objects_dict['classId'] = annot['category_id']
    temp_objects_dict['classTitle'] = coco_file['categories'][annot['category_id'] - 1]['name']

    #coordinates
    coordinate_list = annot['segmentation'][0]

    while len(coordinate_list) > 0:
        xy_list = [coordinate_list.pop(0), coordinate_list.pop(0)]
        temp_objects_dict['points']['exterior'].append(xy_list)

    output_json[image_name]['objects'].append(temp_objects_dict)


In [170]:
#run this to nuke your pc and download all image jsons
for key in output_json.keys():
    with open(f'output/{key}.json', "w") as outfile:
        json.dump(output_json[key], outfile)

In [172]:
#creates meta json
meta_json = {
    "classes": [
        {
            'id': 1, 
            'title': 'Officer',
            "shape": "polygon",
            "hotkey": "O",
            "color": "#FF69B4"
        },
        {
            'id': 2, 
            'title': 'Civilian',
            "shape": "polygon",
            'hotkey': 'C',
            "color": "#008000"
        },
        {
            "id": 3,
            "title": "Riot Shield",
            "shape": "polygon",
            "hotkey": "R",
            "color": "#6FF6F1"
        },
        {
            'id': 4, 
            'title': 'Gun',
            "shape": "polygon",
            'hotkey': 'G',
            'color': '#808080'
        },
        {
            'id': 5, 
            'title': 'Pepper Spray',
            "shape": "polygon",
            'hotkey': 'S',
            "color": "#BF40BF"
        },
        {
            "id": 6,
            "title": "Baton",
            "shape": "polygon",
            "hotkey": "B",
            "color": "#DB490F"
        },
        {
            'id': 7, 
            'title': 'Chemical Smoke',
            "shape": "polygon",
            'hotkey': 'T',
            "color": "#FFBF00"
        }
    ],
    "tags": [
        {
            "id": 22919711,
            "name": "problem",
            "color": "#9F3868",
            "value_type": "none",
            "hotkey": "P",
            "applicable_type": "all",
            "classes": []
        },
    ],
    "projectType": "images"
}

In [173]:
with open(f'output/meta.json', "w") as outfile:
    json.dump(meta_json, outfile)